## Задача X

Напишете клас `SimpleBackup`, който да реализира следните функционалности:

- Създаване на копие на данни по подаден списък с файлове за копиране (`create`). Създаваме копие на всички файлове, но не и на самите директории.
- Възстановяване на копие на данни по подадено име на копие и локация, където да се възстанови (`restore`)
- Връщане на сортиран списък с всички копия на данни, които са направени (`show`)
- Изтриване на копие на данни по подадено име на копие (`delete`)

Име на копие дефинираме по следния начин: `backup_<timestamp>`, където `<timestamp>` е времето на създаване на копието във формат `YYYYMMDD_HHMMSS`.
(За форматирането на датата и часа може да използвате `strftime` от модула `time`.)

Списъка с файлове за копиране е в следния формат - на всеки ред е описан един файл с релативния му път.
Важно: пътищата на файловете са разделени с интервал, а не с `/`.

Копията трябва да се съхраняват в директорията `backups`, като всяко копие е в отделна директория.

Методите `create`, `restore`, `show` и `delete` на копие трябва да връщат `True` ако операцията е била успешна, и `False` в противен случай.

В директорията `task_x` се намира `backups` (където трябва да бъдат създавани вашите копия), списъците с файлове за копиране - `backup_1.txt`, `backup_2.txt` и т.н., и директорията `sample_files`, в която се намират тестовите файлове, на които ще правим копия.

**Бележки:**

Изпълнявайте тетрадката/кода ви в директорията `labs`, за да може да работите с релативни пътища.

Независимо от структурата на входните ви данни, съдържанието на резервните копия трябва да е на едно ниво - т.е всички файлове за дадено копие трябва да се намират в една директория.

In [11]:
# Write your code here:

In [2]:
import os
def get_folder_content(backup_dir: str) -> list[str]:
    result = []

    for content in os.listdir(backup_dir):
        with open(os.path.join(backup_dir, content), 'r') as f:
            result.append(f.read().strip())

    return result

In [13]:
import time
import shutil

backups = SimpleBackup()


# Arrange 
default_listing = backups.show()

# Act 
backup1_result = backups.create(os.path.join('lab04_files', 'task_x', 'backup_1.txt'))
time.sleep(1)  # to ensure that the backups will have a different timestamp
backup2_result = backups.create(os.path.join('lab04_files', 'task_x', 'backup_2.txt'))

listing1_result = backups.show()

backup1_path = listing1_result[-2]
backup2_path = listing1_result[-1]

backup1_content = get_folder_content(os.path.join('lab04_files', 'task_x', 'backups', backup1_path))
backup2_content = get_folder_content(os.path.join('lab04_files', 'task_x', 'backups', backup2_path))

time.sleep(1)  # to ensure that the backups will have a different timestamp

backup3_result = backups.create(os.path.join('lab04_files', 'task_x', 'backup_3.txt'))
time.sleep(1)  # to ensure that the backups will have a different timestamp

backup4_result = backups.create(os.path.join('lab04_files', 'task_x', 'backup_4.txt'))
time.sleep(1)  # to ensure that the backups will have a different timestamp

backup5_result = backups.create(os.path.join('lab04_files', 'task_x', 'backup_5.txt'))
time.sleep(1)  # to ensure that the backups will have a different timestamp

listing2_result = backups.show()

backup3_path = listing2_result[-2]
backup4_path = listing2_result[-1]

backup3_content = get_folder_content(os.path.join('lab04_files', 'task_x', 'backups', backup3_path))
backup4_content = get_folder_content(os.path.join('lab04_files', 'task_x', 'backups', backup4_path))

backup6_result = backups.create(os.path.join('lab04_files', 'task_x', 'backup_4.txt'))
listing3_result = sorted(backups.show())
time.sleep(1)  # to ensure that the backups will have a different timestamp

removal1_result = backups.delete(listing3_result[-1])
listing4_result = backups.show()


restore_results = []
restore_paths = [os.path.join('lab04_files', 'task_x', f'restored_{i}') for i in range(1, 5)]
backups_to_restore = listing4_result[-4:]
restore_contents = []

for backup_to_restore, restore_path in zip(backups_to_restore, restore_paths):
    os.makedirs(restore_path)
    restore_results.append(backups.restore(backup_to_restore, restore_path))
    restore_contents.append(get_folder_content(restore_path))
    shutil.rmtree(restore_path)

restore5_result = backups.restore('backup_20231225_144719', restore_paths[0])
restore6_result = backups.restore('asd', restore_paths[0])

for backup in backups.show():
    backups.delete(backup)

listing5_result = backups.show()

# Tests

# Create
assert backup1_result == True
assert backup2_result == True
assert backup3_result == True
assert backup4_result == True
assert backup5_result == False, 'Config file for backup_5 does not exist'

assert set(backup1_content) == set(['123'])
assert set(backup2_content) == set(['456'])
assert set(backup3_content) == set(['456', '4567'])
assert set(backup4_content) == set(['123', '456', 'Атака, чичо !', '4567'])

# Show
assert len(listing1_result) == len(default_listing) + 2
assert len(listing2_result) == len(default_listing) + 4
assert len(listing3_result) == len(default_listing) + 5
assert len(listing4_result) == len(default_listing) + 4
assert len(listing5_result) == len(default_listing)
assert listing5_result == default_listing

# Restore
assert restore_results[0] == True
assert restore_results[1] == True
assert restore_results[2] == True
assert restore_results[3] == True
assert restore5_result == False, 'Restore of non-existing backup should fail'
assert restore6_result == False, 'Restore of backup with invalid name should fail'


assert set(restore_contents[0]) == set(['123'])
assert set(restore_contents[1]) == set(['456'])
assert set(restore_contents[2]) == set(['456', '4567'])
assert set(restore_contents[3]) == set(['123', '456', 'Атака, чичо !', '4567'])

# Delete
for backup in listing3_result:
    assert os.path.exists(os.path.join('lab04_files', 'task_x', 'backups', backup)) == False


"✅ All OK! +Y points"

[Errno 2] No such file or directory: 'lab04_files/task_x/backup_5.txt'


'✅ All OK! +Y points'